In [ ]:
!pip install -q -U transformers[torch]
!pip install -q -U datasets
!pip install -q -U evaluate
!pip install -q -U tokenizers
!pip install -q -U bitsandbytes
!pip install -q rouge_score
!pip install -q -U peft
!pip install -q -U accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 62.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 38.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 50.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
import re
import random
import numpy as np
from scipy.special import softmax
import pprint

import bitsandbytes as bnb

import torch
import transformers
import evaluate
from datasets import Dataset, load_dataset

# For from-scratch T5 model
from transformers import T5TokenizerFast, T5Config, T5ForConditionalGeneration

# For pre-trained T5 model
from transformers import T5Tokenizer, T5ForConditionalGeneration  # this won't import twice, just noting here what's for each model

# For all T5 models
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq, Trainer, TrainingArguments

# For BLEURT (to load a trained model for evaluation)
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForCausalLM


# For style classifier model (also for evaluating the seq2seq model output)
from transformers import BertTokenizer, BertForSequenceClassification, BitsAndBytesConfig,pipeline
from transformers import TrainingArguments, Trainer

import pandas as pd
from tqdm import tqdm

# Local Data Loading Load Data

In [ ]:
# This cell will authenticate you and mount your Drive in the Colab.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Path to data save in Drive
train = 'FairytaleQA_train.csv'
valid = 'FairytaleQA_valid.csv'
test = 'FairytaleQA_test.csv'
path = 'drive/MyDrive/266_Danielle_Dylan_final_project/data/'
# path = 'drive/MyDrive/DataSci_266/266_Danielle_Dylan_final_project/data/'

In [ ]:
train_filepath = path+train
valid_filepath = path+valid
test_filepath = path+test

In [ ]:
# Save this because we'll need to tell the trainer how many examples we have
num_train_examples = 8_548

In [ ]:
train = load_dataset("csv",data_files=train_filepath,streaming=True)['train']
val = load_dataset("csv",data_files=valid_filepath,streaming=True)['train']
test = load_dataset("csv",data_files=test_filepath,streaming=True)['train']

In [ ]:
train

IterableDataset({
    features: Unknown,
    num_shards: 1
})

In [ ]:
from datasets import load_dataset, DatasetDict

# Combine into a single DatasetDict
ds = DatasetDict({
    "train": train,
    "test": test,
    "validation": val,
})

In [ ]:
train = ds['train'].shuffle()
val = ds['validation'].shuffle()
test = ds['test'].shuffle()

In [ ]:
"""
Initialize the pipeline with bitsandbytes quantization
"""
# Configure bitsandbytes for 4-bit quantization
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
VOCAB_SIZE = 15000
MAX_SEQUENCE_LENGTH = 512

In [ ]:
model_checkpoint = 'Qwen/Qwen2-0.5B-Instruct'

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [ ]:
# We prefix our tasks with "answer the question"
prefix = "Please answer this question with one sentence: "
context = " Context: "

# Define the preprocessing function

def preprocess_function(data):
   """Add prefix to the sentences, tokenize the text, and set the labels"""
   # The "inputs" are the tokenized answer:
   data['full_input'] = [context + story + prefix + question for question,story in zip(data["question"],data['story_section'])]

   input_encoded = tokenizer.batch_encode_plus(data['full_input'],
                      max_length=MAX_SEQUENCE_LENGTH,
                      truncation=True,
                      padding='max_length',
                            return_tensors='pt')

   # The "labels" are the tokenized outputs:
   answers_encoded = tokenizer.batch_encode_plus(data["answer1"],
                      max_length=MAX_SEQUENCE_LENGTH,
                      truncation=True,
                      padding='max_length',
                      return_tensors='pt')

   return {'input_ids': input_encoded['input_ids'],
           'labels': answers_encoded['input_ids']}

In [ ]:
# We prefix our tasks with "answer the question"
prefix = "Please answer this question with one sentence."
context = " Context: "

# Define the preprocessing function

def preprocess_function_qwen_chat(data):
    """Preprocess data using Qwen2 chat format"""

    full_inputs = []
    for question, story, answer in zip(data["question"], data['story_section'], data['answer1']):
        # Format as a conversation
        messages = [
            {"role": "user", "content": f"Context: {story} {prefix}\n\nQuestion: {question}"},
            {"role": "assistant", "content": answer}
        ]

        # Apply chat template
        formatted_text = tokenizer.apply_chat_template(messages, tokenize=False)
        formatted_text = f"User: Context: {story}\n\nQuestion: {question}\nAssistant: {answer}"
        # print(formatted_text)
        full_inputs.append(formatted_text)

    # Tokenize
    tokenized = tokenizer(
        full_inputs,
        max_length=MAX_SEQUENCE_LENGTH,
        truncation=True,
        padding='max_length'
    )

    tokenized['labels'] = tokenized['input_ids'].copy()
    return tokenized

In [ ]:
def create_seq2seq_training_args(batch_size, num_epochs):

    training_args = Seq2SeqTrainingArguments(
        "fairytale_QA_model",
        eval_strategy='epoch',
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=num_epochs,
        report_to='none'

    )

    return training_args

In [ ]:
def create_seq2seq_trainer(model, training_args, train_ds, val_ds):

    trainer = Seq2SeqTrainer(
        model,
        training_args,
        train_dataset=train_ds,
        eval_dataset=val_ds
    )

    return trainer

In [ ]:
def create_training_args(batch_size, num_epochs):

    training_args = TrainingArguments(
        "fairytale_QA_model",
        eval_strategy='epoch',
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=num_epochs,
        report_to='none'

    )

    return training_args

In [ ]:
def create_trainer(model, training_args, train_ds, val_ds):

    trainer = Trainer(
        model,
        training_args,
        train_dataset=train_ds,
        eval_dataset=val_ds
    )

    return trainer

# Fine Tuning Together

In [ ]:
rouge = evaluate.load('rouge')

In [ ]:
# Load the BLEU metric and the trained BLEURT model for semantic similarity scoring

bleu = evaluate.load("bleu")

bleurt_tokenizer = AutoTokenizer.from_pretrained("Elron/bleurt-base-512")
bleurt_model = AutoModelForSequenceClassification.from_pretrained("Elron/bleurt-base-512")

tokenizer_config.json:   0%|          | 0.00/321 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/779 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

# Trying Llama Family on Implicit questions

In [ ]:
from peft import get_peft_config, PeftModel, PeftConfig, get_peft_model, LoraConfig, TaskType
import torch
from transformers import AutoTokenizer

In [ ]:
num_train_examples_im = 2_166

In [ ]:
# Configure bitsandbytes for 4-bit quantization
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
rouge = evaluate.load('rouge')

def compute_metrics_rouge(p):
  predictions, labels = p
  return rouge.compute(predictions=predictions, references=labels)


model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [ ]:
# model_checkpoint = "meta-llama/Meta-Llama-3.1-8B-Instruct"
# model_checkpoint = 'Qwen/Qwen2-1.5B-Instruct'
model_checkpoint = 'Qwen/Qwen2-0.5B-Instruct'
# model_checkpoint = 'Qwen/Qwen2-0.5B'

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

lr = 1e-5
batch_size = 8
num_epochs = 3

In [ ]:
# set LORA specific hyperparameters
r = 8
lora_alpha = 32
lora_dropout = 0.1

In [ ]:
model_im_qwen = AutoModelForCausalLM.from_pretrained(model_checkpoint,quantization_config=quantization_config,
                                                      device_map='auto')

# for name, param in model.named_parameters():
#     param.requires_grad = False

config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

In [ ]:
peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=r,
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout
)

In [ ]:
model_im_qwen = get_peft_model(model_im_qwen, peft_config)
model_im_qwen.print_trainable_parameters()

trainable params: 540,672 || all params: 494,573,440 || trainable%: 0.1093


In [ ]:
_= model_im_qwen.to("cuda")

In [ ]:
# Map the preprocessing function across our dataset
train_im = train.filter(lambda x: x['ex_or_im'] == 'implicit')
val_im = val.filter(lambda x: x['ex_or_im'] == 'implicit')
test_im = test.filter(lambda x: x['ex_or_im'] == 'implicit')

train_tokenized_im = train_im.map(preprocess_function_qwen_chat, batched=True)
val_tokenized_im = val_im.map(preprocess_function_qwen_chat, batched=True)
test_tokenized_im = test_im.map(preprocess_function_qwen_chat, batched=True)

In [ ]:
training_args = TrainingArguments(
    output_dir="qwen-small-lora-text-generation",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_steps=10,
    load_best_model_at_end=True,
    max_steps=int(num_epochs * num_train_examples_im / batch_size),
    report_to='none'
)

In [ ]:
# tokenizer.pad_token = tokenizer.eos_token

In [ ]:
trainer = Trainer(
    model=model_im_qwen,
    args=training_args,
    train_dataset=train_tokenized_im,
    eval_dataset=val_tokenized_im,
    #compute_metrics=compute_metrics_rouge,
)

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
0,1.653000,1.638147
1,1.416300,1.583920
2,1.399500,1.576967


TrainOutput(global_step=812, training_loss=2.231583584118359, metrics={'train_runtime': 2656.7356, 'train_samples_per_second': 2.445, 'train_steps_per_second': 0.306, 'total_flos': 7148496647356416.0, 'train_loss': 2.231583584118359, 'epoch': 2.332512315270936})

In [ ]:
model_qwen_checkpoint_filepath = 'drive/MyDrive/266_Danielle_Dylan_final_project/checkpoints/qwen_model_im_chat'
model_im_qwen.save_pretrained(model_qwen_checkpoint_filepath, from_pt=True)

In [ ]:
# Run this line only if you need to reload the model you trained earlier
model_qwen_checkpoint_filepath = 'drive/MyDrive/266_Danielle_Dylan_final_project/checkpoints/qwen_model_im_chat'
model_im_qwen = AutoModelForCausalLM.from_pretrained(model_qwen_checkpoint_filepath)

In [ ]:
# pipe = pipeline('text-generation',model=model_im_qwen,tokenizer =tokenizer,max_new_tokens=200)

In [ ]:
def generate_qwen_output(model, tokenizer, story, question):
    prefix = "Let's think step by step. Please explain your reasoning before answering."

    messages = [
        {"role": "user", "content": f"Context: {story}\n\nQuestion: {question}\n\n{prefix}"},
        {"role": "assistant", "content": ""}
    ]

    text = tokenizer.apply_chat_template(messages[:-1], tokenize=False, add_generation_prompt=True)
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=100,
        do_sample=True,
        temperature=0.7,
        top_p=0.9
    )

    # Strip out prompt from the output
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return response


In [ ]:
test_im_ds = Dataset.from_list(list(test_im))
test_im_ds

Dataset({
    features: ['story_name', 'story_section', 'question', 'answer1', 'answer2', 'local_or_sum', 'attribute', 'ex_or_im', 'ex_or_im2'],
    num_rows: 253
})

In [ ]:
question = context + test_im_ds['story_section'][0] + prefix + test_im_ds['question'][0]
question

' Context: " what \'s that , tibby ? " said the weaver , with a start as the little cake flew past him . " oh ! " cried she in delight , jumping to her feet , " it is a wee bannock . i wonder where it came from ? " " dinna bother your head about that , tibby , " said her man , " but grip it , my woman , grip it . "Please answer this question with one sentence.what did the weaver\'s husband want her to do ?'

In [ ]:
test_im_ds['question'][1]

'how did maie feel after her husband did not let her buy a cow ?'

In [ ]:
generate_qwen_output(model_im_qwen,tokenizer,test_im_ds['story_section'][1],test_im_ds['question'][1])

'After her husband did not let her buy a cow, Maie felt disappointed. She thought her husband might not want her to keep it or might refuse to give it away at all. Therefore, she felt sad about having to leave Prince alone.'

In [ ]:
# outputs = pipe(question, truncation=True)

In [ ]:
len(question)

406

In [ ]:
test_im_ds['answer2'][1]

'saddened .'

In [ ]:
test_im_ds['answer1'][1]

'sad .'

In [ ]:
%%time

# Store results for aggregate scoring

def process_dataset(dataset):
  results = []
  bleurt_scores = []
  for idx, sample in enumerate(tqdm(dataset)):

      # if idx >= 10:  # Stop after processing 10 samples
      #   return pd.DataFrame(results)
      #   break

      prefix = "Let's think step by step. Please explain your reasoning before answering."
      context = " Context: "

      question = context + sample['story_section'] + prefix + sample['question']

      # Generate summary via the pipeline
      # outputs = pipe(
      #                     question,
      #                     max_new_tokens=50,
      #                     truncation=True,
      #                     temperature=0.6
      # )


      # answer = outputs[0]["generated_text"][len(question):]
      answer = generate_qwen_output(model_im_qwen,tokenizer,sample['story_section'],sample['question'])
      # print(len(answer))
      if (answer.isspace()) | (len(answer) < 1):
        print("\nANSWER IS SPACE for question:",sample['question'],"\nReference:", sample['answer1'])
        print("\nanswer:",[answer])
        answer = 'blank'

      # print("\nbegin answer:", answer, ":end answer")
      # print("\n",[answer],"****")

      # Evaluation
      predictions = [answer]
      reference1 = [sample['answer1']]
      reference2 = [sample['answer2']]

      # Calculate ROUGE scores
      rouge_scores1 = rouge.compute(predictions=predictions, references=reference1)
      if reference1 != reference2:
        rouge_scores2 = rouge.compute(predictions=predictions, references=reference2)
      else:
        rouge_scores2 = rouge_scores1
      rouge_scores1 = {k+'_a1': v for k, v in rouge_scores1.items()}
      rouge_scores2 = {k+'_a2': v for k, v in rouge_scores2.items()}


      # Calculate BLEU scores
      bleu_scores1 = bleu.compute(predictions=predictions, references=reference1,smooth=True)
      if reference1 != reference2:
        bleu_scores2 = bleu.compute(predictions=predictions, references=reference2,smooth=True)
      else:
        bleu_scores2 = bleu_scores1
      bleu_scores1 = {k+'_a1': v for k, v in bleu_scores1.items()}
      bleu_scores2 = {k+'_a2': v for k, v in bleu_scores2.items()}

      # Calculate BLEURT scores
      with torch.no_grad():
        bleurt_score_a1 = bleurt_model(**bleurt_tokenizer(answer,
                                                sample['answer1'],
                                                truncation=True,
                                                max_length=512,
                                                padding='max_length',
                                                return_tensors='pt'))[0].squeeze().numpy()
        if reference1 != reference2:
          bleurt_score_a2 = bleurt_model(**bleurt_tokenizer(answer,
                                                sample['answer2'],
                                                truncation=True,
                                                max_length=512,
                                                padding='max_length',
                                                return_tensors='pt'))[0].squeeze().numpy()
        else:
          bleurt_score_a2 = bleurt_score_a1


      # Store results
      results.append({
          'id': idx,
          'question': sample['question'],
          'ex_or_im': sample['ex_or_im'],
          'story_section': sample['story_section'][:500],  # Store truncated text for readability
          'reference_answer1': sample['answer1'],
          'reference_answer2': sample['answer2'],
          'generated_answer': answer,
          **rouge_scores1,
          **rouge_scores2,
          **bleu_scores1,
          **bleu_scores2,
          'bleurt_score_a1': bleurt_score_a1,
          'bleurt_score_a2': bleurt_score_a2,
      })

      # Print progress update every 10 samples
      if (idx + 1) % 100 == 0:
          print(f"\nProcessed {idx + 1} samples; Results length: {len(results)}")
          print(f"Latest Answer1 ROUGE-1: {rouge_scores1['rouge1_a1']:.4f}")

  return pd.DataFrame(results)

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 9.06 µs


In [ ]:
test_im_ds = Dataset.from_list(list(test_im))
test_results_im_df = process_dataset(test_im_ds)

  0%|          | 0/253 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
 40%|███▉      | 100/253 [32:54<39:26, 15.47s/it]


Processed 100 samples; Results length: 100
Latest Answer1 ROUGE-1: 0.0625


 79%|███████▉  | 200/253 [1:07:29<16:11, 18.34s/it]


Processed 200 samples; Results length: 200
Latest Answer1 ROUGE-1: 0.0000


100%|██████████| 253/253 [1:26:38<00:00, 20.55s/it]


In [ ]:
test_results_im_df.head()

,id,question,ex_or_im,story_section,reference_answer1,reference_answer2,generated_answer,rouge1_a1,rouge2_a1,rougeL_a1,...,translation_length_a1,reference_length_a1,bleu_a2,precisions_a2,brevity_penalty_a2,length_ratio_a2,translation_length_a2,reference_length_a2,bleurt_score_a1,bleurt_score_a2
0,0,what did the weaver's husband want her to do ?,implicit,""" what 's that , tibby ? "" said the weaver , w...",grab the bannock .,grip it .,The weaver's husband wanted her to hold the sm...,0.071429,0.000000,0.071429,...,26,4,0.051660,"[0.1111111111111111, 0.038461538461538464, 0.0...",1.0,8.666667,26,3,-1.8227786,-1.575173
1,1,how did maie feel after her husband did not le...,implicit,' what would you do with a cow ? ' asked matte...,sad .,saddened .,"After her husband did not let her buy a cow, M...",0.000000,0.000000,0.000000,...,49,2,0.024526,"[0.04, 0.02040816326530612, 0.0208333333333333...",1.0,24.500000,49,2,-1.3935102,-1.2800059
2,2,what did the tailor's wife and the three tailo...,implicit,"now , it chanced that this house belonged to a...",eat the bannock .,split it between them and drink milk .,The tailor's wife wanted to take the bannock h...,0.100000,0.052632,0.100000,...,40,4,0.047138,"[0.14634146341463414, 0.05, 0.0256410256410256...",1.0,5.000000,40,8,-1.4434128,-1.506901
3,3,why did all three daughters have to spend the ...,implicit,then dullhead fell to at once to hew down the ...,they were all stuck to the goose .,they were stuck to the goose .,All three daughters had to spend the night wit...,0.079208,0.020202,0.079208,...,99,8,0.018055,"[0.05, 0.020202020202020204, 0.010204081632653...",1.0,14.142857,99,7,-1.5643634,-1.6135697
4,4,what did the nurse long for ?,implicit,"' the king of granada is my father , and i was...",the princess 's wrinkles to disappear .,that the princess ' wrinkled skin should disap...,the nurse wished for the prince to die. \n\nre...,0.063158,0.000000,0.063158,...,97,7,0.017429,"[0.04081632653061224, 0.020618556701030927, 0....",1.0,10.777778,97,9,-1.6507529,-1.4620209


In [ ]:
test_results_im_df.tail(10)

,id,question,ex_or_im,story_section,reference_answer1,reference_answer2,generated_answer,rouge1_a1,rouge2_a1,rougeL_a1,...,translation_length_a1,reference_length_a1,bleu_a2,precisions_a2,brevity_penalty_a2,length_ratio_a2,translation_length_a2,reference_length_a2,bleurt_score_a1,bleurt_score_a2
243,243,how did the skillful fisher feel to see his br...,implicit,now the skillful fisher had used the lost hook...,surprised .,very angry .,The skillful fisher felt relieved to see his b...,0.000000,0.000000,0.000000,...,95,2,0.012585,"[0.020833333333333332, 0.010526315789473684, 0...",1.0,31.666667,95,3,-1.5537467,-1.6890903
244,244,how did the gentleman feel after he realized h...,implicit,"left to himself , the full horror of his comin...",surprised .,impressed .,The gentleman felt relieved that the old woman...,0.000000,0.000000,0.000000,...,100,2,0.011953,"[0.019801980198019802, 0.01, 0.010101010101010...",1.0,50.000000,100,2,-1.6350656,-1.6504788
245,245,why did the leader punish the four men who wer...,implicit,"' trust me and eat as you did before , and no ...",he found out ian was still alive .,he thinks they did not kill ian since he is st...,The leader punished the four men who were told...,0.076923,0.000000,0.076923,...,49,8,0.039894,"[0.14, 0.04081632653061224, 0.0208333333333333...",1.0,3.769231,49,13,-0.91565514,-0.5359696
246,246,why did the man get back in the tree ?,implicit,"but on looking out from the tree , what was hi...",he did not want to be seen by the demons .,he did not want to be seen by the demons .,The man got back in the tree because it was sa...,0.208333,0.043478,0.208333,...,41,11,0.047780,"[0.16666666666666666, 0.04878048780487805, 0.0...",1.0,3.727273,41,11,-0.44960687,-0.44960687
247,247,how did matte know there was going to be a sto...,implicit,"about midnight the fisherman sat up , and said...",he heard the weathercock .,the weather - cock .,"Matte knew that the weathercock was screaming,...",0.114286,0.060606,0.114286,...,34,5,0.039308,"[0.08571428571428572, 0.029411764705882353, 0....",1.0,6.800000,34,5,-0.88672745,-1.6185112
248,248,how did the king feel about dullhead as a son-...,implicit,after a time they all came to a town where a k...,unhappy .,upset .,The king didn't like dullhead because he thoug...,0.000000,0.000000,0.000000,...,40,2,0.030119,"[0.04878048780487805, 0.025, 0.025641025641025...",1.0,20.000000,40,2,-1.6367844,-1.837249
249,249,how did countess fraukirk feel after she heard...,implicit,"but the wily countess offered him much gold , ...",content .,impressed .,"After hearing snorro's plan, Countess Fraukirk...",0.000000,0.000000,0.000000,...,55,2,0.021825,"[0.03571428571428571, 0.01818181818181818, 0.0...",1.0,27.500000,55,2,-1.7495487,-1.9405919
250,250,why did snorro choose to live in the dwarfie s...,implicit,but all the time they never guessed the real r...,he wanted to find a stone .,he wanted to find the carbuncle .,snorro chose to live in the dwarfie stone beca...,0.108696,0.022222,0.108696,...,94,7,0.019907,"[0.06315789473684211, 0.02127659574468085, 0.0...",1.0,13.428571,94,7,-1.1882038,-1.5854493
251,251,what will happen because the jelly fish asked ...,implicit,"thus they went along , the jelly fish skimming...",the monkey will become suspicious .,the monkey will get suspicious .,The jelly fish will ask the monkey what he nee...,0.063158,0.021505,0.042105,...,97,6,0.018429,"[0.05102040816326531, 0.020618556701030927, 0....",1.0,16.166667,97,6,-1.139472,-1.2295055
252,252,how will snorro the dwarf poison paul ?,implicit,"but the wily countess offered him much gold , ...",poison his garment .,he will make a poison cloth that is very beaut...,"snorro the dwarf intends to poison paul, who i...",0.023529,0.000000,0.023529,...,91,4,0.034590,"[0.13043478260869565, 0.04395604395604396, 0.0...",1.0,4.789474,91,19,-1.4268811,-0.9401994


In [ ]:
test_results_im_df.to_csv('drive/MyDrive/266_Danielle_Dylan_final_project/answer_outputs/qwen_finetuned_imOnly_chat_CoT.csv',index=False)

In [ ]:
# def generate_output(model, tokenizer, data, **kwargs):

#     all_outputs = []
#     prefix = "Please answer this question: "
#     context = " Context: "

#     input_sentences = [prefix + sample['question'] + context + sample['story_section'] for sample in data]

#     for i in range(int(len(input_sentences)) + 1):
#         if i >= len(input_sentences):
#             break

#         inputs_encoded = tokenizer(input_sentences[i],truncation=True, padding=True, return_tensors='pt')
#         output_ids = model.cuda().generate(input_ids=inputs_encoded['input_ids'].cuda(), **kwargs)
#         generated_sentences = tokenizer.batch_decode(output_ids,
#                                                      skip_special_tokens=True,
#                                                      clean_up_tokenization_spaces=False)

#         all_outputs.extend(generated_sentences)

#     return all_outputs

In [ ]:
# def calculate_eval_metrics(data, model, tokenizer, prefix="", **kwargs):

#     # Translate original texts
#     answers = generate_output(model, tokenizer, data, **kwargs)
#     references = [example['answer1'] for example in data]

#     # Calculate ROUGE scores
#     rouge_results = rouge.compute(predictions=answers, references=references)
#     print('ROUGE: ', rouge_results)

#     # Calculate BLEURT scores
#     bleurt_scores = []
#     for i in range(int(len(answers)) + 1):
#         if i >= len(answers):
#             break

#         with torch.no_grad():
#             scores = bleurt_model(**bleurt_tokenizer(references[i],
#                                                      answers[i],
#                                                      truncation=True,
#                                                      max_length=MAX_SEQUENCE_LENGTH,
#                                                      padding='max_length',
#                                                      return_tensors='pt'))[0].squeeze().numpy()
#             if scores.shape:
#                 bleurt_scores.extend(scores)
#             else:  # Happens when there was only one example in the last batch
#                 bleurt_scores.append(float(scores))

#     print('BLEURT: ', np.mean(bleurt_scores))

#     return answers

In [ ]:
# """
# Playing with the decoder .generate() arguments like num_beams or top_p, etc.
# """

# generate_kwargs = {

#     'num_beams': 4,
#     'do_sample':True,
#     'top_k':200,
#     'top_p':0.95,
#     'temperature':0.4,
#     'no_repeat_ngram_size':4,
#     'max_new_tokens': 500
# }

# test_im_list = list(test_im.take(100))  # or however many you evaluated

# test_answers_im = calculate_eval_metrics(
#     # test_tokenized_im,
#     test_im_list,
#     model_im_llama,
#     tokenizer,
#     batch_size,
#     **generate_kwargs
# )

In [ ]:
# # Print out a sample of outputs to manually review

# for i in range(10):
#     sample_i = random.choice(range(len(test_im_list)))
#     print('Referenced answer:  ', test_im_list[i]['answer1'])
#     print('Generated answer:   ', test_answers_im[i])
#     print()

In [ ]:
# def clear_gpu_memory():
#     import gc
#     import torch

#     gc.collect()
#     torch.cuda.empty_cache()
#     print("Cleared GPU memory.")

# # Usage
# del model_im_cot, tokenizer, trainer
# clear_gpu_memory()


In [ ]:
def generate_qwen_output(model,tokenizer,story,question):


  messages = [
      {"role": "user", "content": f"Context: {story} {prefix}\n\nQuestion: {question}"},
      {"role": "assistant", "content": ""}
  ]
  text = tokenizer.apply_chat_template(
      messages[:-1],
      tokenize=False,
      add_generation_prompt=True
  )
  model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

  generated_ids = model.generate(
      **model_inputs,
      max_new_tokens=50,
      # pad_token_id=tokenizer.eos_token_id
  )
  generated_ids = [
      output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
  ]

  response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
  return response

In [ ]:
# We prefix our tasks with "answer the question"
prefix = "Please answer this question with one sentence."
context = " Context: "

# Define the preprocessing function

def preprocess_function_qwen_chat(data):
    """Preprocess data using Qwen2 chat format"""

    full_inputs = []
    for question, story, answer in zip(data["question"], data['story_section'], data['answer1']):
        # Format as a conversation
        messages = [
            {"role": "user", "content": f"Context: {story} {prefix}\n\nQuestion: {question}"},
            {"role": "assistant", "content": answer}
        ]

        # Apply chat template
        formatted_text = tokenizer.apply_chat_template(messages, tokenize=False)
        formatted_text = f"User: Context: {story}\n\nQuestion: {question}\nAssistant: {answer}"
        # print(formatted_text)
        full_inputs.append(formatted_text)

    # Tokenize
    tokenized = tokenizer(
        full_inputs,
        max_length=MAX_SEQUENCE_LENGTH,
        truncation=True,
        padding='max_length'
    )

    tokenized['labels'] = tokenized['input_ids'].copy()
    return tokenized